In [47]:
from importlib import reload
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
sns.set_theme(style='whitegrid')
import ard_logging
import logging
ard_logging.init(force_debug=True)
LOG = logging.getLogger(__name__)
LOG.info('Hello from notebook')

df = sns.load_dataset('diamonds')
df.info()
df.head()

2022-06-28 13:23:36,842:    INFO:      ard_logging: MainProcess 148: Logging initialized.
2022-06-28 13:23:36,843:    INFO:         __main__: MainProcess 148: Hello from notebook


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 53940 entries, 0 to 53939
Data columns (total 10 columns):
 #   Column   Non-Null Count  Dtype   
---  ------   --------------  -----   
 0   carat    53940 non-null  float64 
 1   cut      53940 non-null  category
 2   color    53940 non-null  category
 3   clarity  53940 non-null  category
 4   depth    53940 non-null  float64 
 5   table    53940 non-null  float64 
 6   price    53940 non-null  int64   
 7   x        53940 non-null  float64 
 8   y        53940 non-null  float64 
 9   z        53940 non-null  float64 
dtypes: category(3), float64(6), int64(1)
memory usage: 3.0 MB


,carat,cut,color,clarity,depth,table,price,x,y,z
0,0.23,Ideal,E,SI2,61.5,55.0,326,3.95,3.98,2.43
1,0.21,Premium,E,SI1,59.8,61.0,326,3.89,3.84,2.31
2,0.23,Good,E,VS1,56.9,65.0,327,4.05,4.07,2.31
3,0.29,Premium,I,VS2,62.4,58.0,334,4.20,4.23,2.63
4,0.31,Good,J,SI2,63.3,58.0,335,4.34,4.35,2.75


In [96]:
import ard_fit_pipelines as fp
reload(fp.core); reload(fp.transforms); reload(fp)
import ard_fit_pipelines.core as fpc
import ard_fit_pipelines.transforms as fpt

logging.getLogger('ard_fit_pipelines').setLevel(logging.DEBUG)
#reload(fpc)

In [97]:
cols = ['price', 'x', 'y' ,'z']
t = fpt.DeMean(cols)

In [98]:
t

DeMean(cols=['price', 'x', 'y', 'z'], w_col=None)

In [92]:
fpt.DeMean?

Init signature:
fpt.DeMean(
    cols: 'str | HP | Iterable[str | HP]',
    w_col: 'Optional[str]' = None,
) -> None
Docstring:      De-mean some columns.
Init docstring: Method generated by attrs for class DeMean.
File:           ~/projects/ardua/codebases/local/ard-fit-pipelines/src/ard_fit_pipelines/transforms.py
Type:           ABCMeta
Subclasses:     


In [82]:
class DeMean(fpc.WeightedTransform, fpc.ColumnsTransform):
    """
    De-mean some columns.
    """

    def _fit(self, df_fit: pd.DataFrame) -> object:
        if self.w_col is not None:
            return weighted_means(df_fit, self.cols, self.w_col)
        return df_fit[self.cols].mean()

    def _apply(self, df_apply: pd.DataFrame, state: pd.DataFrame):
        means = state
        return df_apply.assign(**{c: df_apply[c] - means[c] for c in self.cols})
    
DeMean?

Init signature: DeMean(w_col: 'Optional[str]' = None) -> None
Docstring:      De-mean some columns.
Init docstring: Method generated by attrs for class WeightedTransform.
File:           /usr/lib/python3.9/abc.py
Type:           ABCMeta
Subclasses:     


In [89]:
from attrs import define
@define(slots=False)
class Foo:
    foo: int
    
@define(slots=False)
class Bar:
    bar: int
    
@define
class Baz(Foo, Bar):
    pass

Baz?

Init signature: Baz(bar: int, foo: int) -> None
Docstring:      <no docstring>
Init docstring: Method generated by attrs for class Baz.
Type:           type
Subclasses:     


In [63]:

df[['depth', 'table']].multiply(df['carat'], axis='index').sum() / df['carat'].sum()

depth    61.773424
table    57.698256
dtype: float64

In [57]:
df.multiply?

Signature: df.multiply(other, axis='columns', level=None, fill_value=None)
Docstring:
Get Multiplication of dataframe and other, element-wise (binary operator `mul`).

Equivalent to ``dataframe * other``, but with support to substitute a fill_value
for missing data in one of the inputs. With reverse version, `rmul`.

Among flexible wrappers (`add`, `sub`, `mul`, `div`, `mod`, `pow`) to
arithmetic operators: `+`, `-`, `*`, `/`, `//`, `%`, `**`.

Parameters
----------
other : scalar, sequence, Series, or DataFrame
    Any single or multiple element data structure, or list-like object.
axis : {0 or 'index', 1 or 'columns'}
    Whether to compare by the index (0 or 'index') or columns
    (1 or 'columns'). For Series input, axis to match Series index on.
level : int or label
    Broadcast across a level, matching Index values on the
    passed MultiIndex level.
fill_value : float or None, default None
    Fill existing missing (NaN) values, and any new element needed for
    successful Dat

In [52]:
lambda_demean = fpt.StatefulLambda(
        fit_fun=lambda df: df["price"].mean(),
        apply_fun=lambda df, mean: df.assign(price=df["price"] - mean),
    )
lambda_add_ones = fpt.StatelessLambda(
    apply_fun=lambda df: df.assign(ones=1.0)
)
result = fpc.IfHyperparamIsTrue('do_it', then = lambda_demean).fit(df, bindings={'do_it': False}).apply(df)
result = fpc.IfHyperparamIsTrue('do_it', then = lambda_demean).fit(df, bindings={'do_it': True}).apply(df)
result = fpc.IfHyperparamIsTrue('do_it', then=lambda_demean, otherwise=lambda_add_ones).fit(df, bindings={'do_it': False}).apply(df)
result = fpc.IfHyperparamIsTrue('do_it', then=lambda_add_ones, otherwise=lambda_demean).fit(df, bindings={'do_it': False}).apply(df)
result.head()

2022-06-28 13:25:01,842:   DEBUG: ard_fit_pipelines.core: MainProcess 148: Fitting IfHyperparamIsTrue on 53940 rows: IfHyperparamIsTrue(name='do_it', then=StatefulLambda(fit_fun=<function <lambda> at 0x7f9e503ddf70>, apply_fun=<function <lambda> at 0x7f9e503dde50>), otherwise=None)
2022-06-28 13:25:01,844:   DEBUG: ard_fit_pipelines.core: MainProcess 148: Applying IfHyperparamIsTrue.FitIfHyperparamIsTrue to 53940 rows: FitIfHyperparamIsTrue(name='do_it', then=StatefulLambda(fit_fun=<function <lambda> at 0x7f9e503ddf70>, apply_fun=<function <lambda> at 0x7f9e503dde50>), otherwise=None, <53940 rows of fitting data>)
2022-06-28 13:25:01,844:   DEBUG: ard_fit_pipelines.core: MainProcess 148: Fitting IfHyperparamIsTrue on 53940 rows: IfHyperparamIsTrue(name='do_it', then=StatefulLambda(fit_fun=<function <lambda> at 0x7f9e503ddf70>, apply_fun=<function <lambda> at 0x7f9e503dde50>), otherwise=None)
2022-06-28 13:25:01,845:   DEBUG: ard_fit_pipelines.core: MainProcess 148: Fitting StatefulLamb

,carat,cut,color,clarity,depth,table,price,x,y,z
0,0.23,Ideal,E,SI2,61.5,55.0,-3606.799722,3.95,3.98,2.43
1,0.21,Premium,E,SI1,59.8,61.0,-3606.799722,3.89,3.84,2.31
2,0.23,Good,E,VS1,56.9,65.0,-3605.799722,4.05,4.07,2.31
3,0.29,Premium,I,VS2,62.4,58.0,-3598.799722,4.20,4.23,2.63
4,0.31,Good,J,SI2,63.3,58.0,-3597.799722,4.34,4.35,2.75


In [30]:
fpt.StatelessLambda(lambda df: df.rename(columns={'price': 'price_orig'})).apply(df).head()

2022-06-28 11:14:11,340:   DEBUG: ard_fit_pipelines.core: MainProcess 148: Fitting StatelessLambda on 0 rows: StatelessLambda(lambda_fun=<function <lambda> at 0x7f9e5021c430>)
2022-06-28 11:14:11,341:   DEBUG: ard_fit_pipelines.core: MainProcess 148: Applying StatelessLambda.FitStatelessLambda to 53940 rows: FitStatelessLambda(lambda_fun=<function <lambda> at 0x7f9e5021c430>, <0 rows of fitting data>)


,carat,cut,color,clarity,depth,table,price_orig,x,y,z
0,0.23,Ideal,E,SI2,61.5,55.0,326,3.95,3.98,2.43
1,0.21,Premium,E,SI1,59.8,61.0,326,3.89,3.84,2.31
2,0.23,Good,E,VS1,56.9,65.0,327,4.05,4.07,2.31
3,0.29,Premium,I,VS2,62.4,58.0,334,4.20,4.23,2.63
4,0.31,Good,J,SI2,63.3,58.0,335,4.34,4.35,2.75


In [34]:
fpt.StatelessLambda(
    lambda df, bindings: df.rename(columns={bindings['response']: 'foobagger'})
).apply(df, bindings={'response': 'price'}).head()

2022-06-28 11:15:24,559:   DEBUG: ard_fit_pipelines.core: MainProcess 148: Fitting StatelessLambda on 0 rows: StatelessLambda(lambda_fun=<function <lambda> at 0x7f9e50117040>)
2022-06-28 11:15:24,561:   DEBUG: ard_fit_pipelines.core: MainProcess 148: Applying StatelessLambda.FitStatelessLambda to 53940 rows: FitStatelessLambda(lambda_fun=<function <lambda> at 0x7f9e50117040>, <0 rows of fitting data>)


,carat,cut,color,clarity,depth,table,foobagger,x,y,z
0,0.23,Ideal,E,SI2,61.5,55.0,326,3.95,3.98,2.43
1,0.21,Premium,E,SI1,59.8,61.0,326,3.89,3.84,2.31
2,0.23,Good,E,VS1,56.9,65.0,327,4.05,4.07,2.31
3,0.29,Premium,I,VS2,62.4,58.0,334,4.20,4.23,2.63
4,0.31,Good,J,SI2,63.3,58.0,335,4.34,4.35,2.75


In [43]:
df = pd.DataFrame({
    'col1': pd.Series([1.0, np.nan, 2.0])
})
df

df = pd.DataFrame({
        'col1': [float(x) for x in range(1, 101)]
    })
df.quantile(0.2, interpolation='nearest'), df.quantile(0.8, interpolation='nearest')

(col1    21.0
 Name: 0.2, dtype: float64,
 col1    80.0
 Name: 0.8, dtype: float64)

In [42]:
df.quantile?

Signature:
df.quantile(
    q=0.5,
    axis: 'Axis' = 0,
    numeric_only: 'bool' = True,
    interpolation: 'str' = 'linear',
)
Docstring:
Return values at the given quantile over requested axis.

Parameters
----------
q : float or array-like, default 0.5 (50% quantile)
    Value between 0 <= q <= 1, the quantile(s) to compute.
axis : {0, 1, 'index', 'columns'}, default 0
    Equals 0 or 'index' for row-wise, 1 or 'columns' for column-wise.
numeric_only : bool, default True
    If False, the quantile of datetime and timedelta data will be
    computed as well.
interpolation : {'linear', 'lower', 'higher', 'midpoint', 'nearest'}
    This optional parameter specifies the interpolation method to use,
    when the desired quantile lies between two data points `i` and `j`:

    * linear: `i + (j - i) * fraction`, where `fraction` is the
      fractional part of the index surrounded by `i` and `j`.
    * lower: `i`.
    * higher: `j`.
    * nearest: `i` or `j` whichever is nearest.
    * mi

In [4]:
t = fpt.DeMean(cols=fpc.HP('response_cols'))
tfit = t.fit(df, bindings={'response_cols':['price']})
tfit.apply(df).head()

fpt.CopyColumns(cols=['price'], dest_cols=['price_copy_1', 'price_copy_2']).apply(df).head()

2022-06-28 09:34:40,690:   DEBUG: ard_fit_pipelines.core: MainProcess 148: Fitting DeMean on 53940 rows: DeMean(cols=HP(name='response_cols'))
2022-06-28 09:34:40,694:   DEBUG: ard_fit_pipelines.core: MainProcess 148: Applying DeMean.FitDeMean to 53940 rows: FitDeMean(cols=['price'], <53940 rows of fitting data>)
2022-06-28 09:34:40,697:   DEBUG: ard_fit_pipelines.core: MainProcess 148: Fitting CopyColumns on 0 rows: CopyColumns(cols=['price'], dest_cols=['price_copy_1', 'price_copy_2'])
2022-06-28 09:34:40,698:   DEBUG: ard_fit_pipelines.core: MainProcess 148: Applying CopyColumns.FitCopyColumns to 53940 rows: FitCopyColumns(cols=['price'], dest_cols=['price_copy_1', 'price_copy_2'], <0 rows of fitting data>)


,carat,cut,color,clarity,depth,table,price,x,y,z,price_copy_1,price_copy_2
0,0.23,Ideal,E,SI2,61.5,55.0,326,3.95,3.98,2.43,326,326
1,0.21,Premium,E,SI1,59.8,61.0,326,3.89,3.84,2.31,326,326
2,0.23,Good,E,VS1,56.9,65.0,327,4.05,4.07,2.31,327,327
3,0.29,Premium,I,VS2,62.4,58.0,334,4.20,4.23,2.63,334,334
4,0.31,Good,J,SI2,63.3,58.0,335,4.34,4.35,2.75,335,335


In [23]:
f = lambda df, b=None: df.rename(lambda c: c+'_foo')

In [25]:
import inspect
dir(inspect.signature(f))
inspect.signature(f).parameters

mappingproxy({'df': <Parameter "df">, 'b': <Parameter "b=None">})

In [30]:
cols = ['price', 'x', 'y', 'z']
t = fpt.DeMean(cols)
result = t.fit(df).apply(df)
(result[cols].mean().abs() < 1e-10).all()

2022-06-20 13:57:00,151:   DEBUG: ard_fit_pipelines.core: MainProcess 3343: Fitting DeMean on 53940 rows: DeMean(cols=['price', 'x', 'y', 'z'])
2022-06-20 13:57:00,156:   DEBUG: ard_fit_pipelines.core: MainProcess 3343: Applying DeMean.FitDeMean to 53940 rows: FitDeMean(cols=['price', 'x', 'y', 'z'], <53940 rows of fitting data>)


cols: Bound ['price', 'x', 'y', 'z'] -> ['price', 'x', 'y', 'z']


True

In [19]:
class DeMean(fpc.ColumnsTransform):
    "Docstr for MyDeMean"
    
    def _fit(self, X_fit):
        return X_fit[self.cols].mean()
    
    def _apply(self, X_apply, state):
        means = state
        return X_apply.assign(**{
            c: X_apply[c] - means[c]
            for c in self.cols
        })
        

#t = DeMean(cols=['price'])
t = DeMean(cols=fpc.hp('response_cols'))
tfit = t.fit(df, bindings={'response_cols':['price']})
tfit.apply(df).head()

2022-06-20 13:46:03,738:   DEBUG: ard_fit_pipelines.core: MainProcess 3343: Fitting DeMean on 53940 rows: DeMean(cols=hp(name='response_cols'))
2022-06-20 13:46:03,748:   DEBUG: ard_fit_pipelines.core: MainProcess 3343: Applying DeMean.FitDeMean to 53940 rows: FitDeMean(cols=['price'], <53940 rows of fitting data>)


cols: Bound hp(name='response_cols') -> ['price']


,carat,cut,color,clarity,depth,table,price,x,y,z
0,0.23,Ideal,E,SI2,61.5,55.0,-3606.799722,3.95,3.98,2.43
1,0.21,Premium,E,SI1,59.8,61.0,-3606.799722,3.89,3.84,2.31
2,0.23,Good,E,VS1,56.9,65.0,-3605.799722,4.05,4.07,2.31
3,0.29,Premium,I,VS2,62.4,58.0,-3598.799722,4.20,4.23,2.63
4,0.31,Good,J,SI2,63.3,58.0,-3597.799722,4.34,4.35,2.75


In [24]:
repr(t)

"DeMean(cols=hp(name='response_cols'))"

In [25]:
repr(tfit)

"FitDeMean(cols=['price'], <53940 rows of fitting data>)"

In [23]:
isinstance(DeMean.FitDeMean, type) 

True

In [177]:
t = fpc.Identity()
t.apply(df).head()

2022-06-20 12:10:52,908:   DEBUG: ard_fit_pipelines.core: MainProcess 2337: Fitting Identity on 0 rows: Identity()
2022-06-20 12:10:52,909:   DEBUG: ard_fit_pipelines.core: MainProcess 2337: Applying Identity.FitIdentity to 53940 rows: FitIdentity(<0 rows of fitting data>)


,carat,cut,color,clarity,depth,table,price,x,y,z
0,0.23,Ideal,E,SI2,61.5,55.0,326,3.95,3.98,2.43
1,0.21,Premium,E,SI1,59.8,61.0,326,3.89,3.84,2.31
2,0.23,Good,E,VS1,56.9,65.0,327,4.05,4.07,2.31
3,0.29,Premium,I,VS2,62.4,58.0,334,4.20,4.23,2.63
4,0.31,Good,J,SI2,63.3,58.0,335,4.34,4.35,2.75


In [137]:
help(DeMean.FitDeMean)

Help on class FitDeMean in module ard_fit_pipelines.core:

class FitDeMean(FitTransform)
 |  FitDeMean(transform: 'DeMean', X_fit: 'pd.DataFrame', bindings=None)
 |  
 |  The result of fitting a DeMean Transform. Blah blah.
 |  
 |  Method resolution order:
 |      FitDeMean
 |      FitTransform
 |      abc.ABC
 |      builtins.object
 |  
 |  Data and other attributes defined here:
 |  
 |  __abstractmethods__ = frozenset()
 |  
 |  ----------------------------------------------------------------------
 |  Methods inherited from FitTransform:
 |  
 |  __init__(self, transform: 'DeMean', X_fit: 'pd.DataFrame', bindings=None)
 |      Docstr for FitTransform.__init__
 |  
 |  __repr__(self)
 |      Return repr(self).
 |  
 |  apply(self, X_apply: 'pd.DataFrame') -> 'pd.DataFrame'
 |      Docstr for FitTransform.apply
 |  
 |  state(self)
 |  
 |  ----------------------------------------------------------------------
 |  Class methods inherited from FitTransform:
 |  
 |  __init_subclass_

In [10]:
DeMean.fit?

Signature:
DeMean.fit(
    self,
    X_fit: 'pd.DataFrame',
    bindings: 'Optional[dict[str, object]]' = None,
) -> 'DeMean.FitDeMean'
Docstring: <no docstring>
File:      ~/projects/ardua/codebases/local/ard-fit-pipelines/src/ard_fit_pipelines/core.py
Type:      function


In [13]:
tfit